<h4>Obtener datos de canciones de AcousticBrainz según su recording_id 🎵</h4>

## Resumen del Proceso de Extracción y Procesamiento de Datos de AcousticBrainz 🎵

### Objetivo 🎯
El objetivo de este notebook es obtener datos de canciones desde AcousticBrainz utilizando sus `recording_id`, procesar estos datos y almacenarlos en archivos CSV para su posterior análisis.

### Pasos del Proceso 📝

1. **Importación de Módulos y Carga de Datos Iniciales** 📥
    - Se importan los módulos necesarios: `numpy`, `pandas`, `requests`, `time` y `os`.
    - Se carga un archivo CSV que contiene los `recording_id` de las canciones que queremos procesar.

2. **Definición de Rutas de Datos** 🗺️
    - Se define un diccionario `dataRoute` que contiene las rutas de los datos de interés dentro de la estructura JSON devuelta por AcousticBrainz.

3. **Función para Obtener Datos desde AcousticBrainz** 🔍
    - Se define la función `get_data_by_recording_id` que realiza solicitudes a la API de AcousticBrainz utilizando el `recording_id`. La función incluye lógica para manejar errores y reintentos en caso de que se excedan los límites de tasa de la API.

4. **Inicialización de Contadores y Archivos de Salida** 🗃️
    - Se inicializan contadores para monitorear el estado de las solicitudes y el procesamiento de filas.
    - Se definen los archivos de salida para almacenar los datos procesados y los errores. Si los archivos no existen, se crean con los encabezados correspondientes.

5. **Procesamiento de Datos** ⚙️
    - Se itera sobre cada fila del dataset inicial, obteniendo el `recording_id` y utilizando la función `get_data_by_recording_id` para obtener los datos desde AcousticBrainz.
    - Se procesan los datos obtenidos y se asignan a las columnas correspondientes en el dataset.
    - Se manejan los errores y se registran los `recording_id` que no se pudieron procesar.
    - Se escribe cada fila procesada en el archivo de salida correspondiente.

6. **Carga y Análisis de Datos Procesados** 📊
    - Se carga un archivo CSV con los datos procesados y se muestra información sobre el dataset.
    - Se seleccionan los primeros registros procesados y se guardan en un nuevo archivo CSV.
    - Se eliminan los registros ya procesados del dataset original y se guardan los datos restantes en un nuevo archivo CSV para continuar el procesamiento en futuras ejecuciones.

### Resultados 📈
Al final del proceso, se obtienen dos archivos CSV:
- `processed_data_songs.csv`: Contiene los datos de las canciones que se han procesado exitosamente.
- `not_processed_songs.csv`: Contiene los datos de las canciones que aún no se han procesado.

Este proceso permite obtener y estructurar datos de AcousticBrainz de manera eficiente, manejando errores y límites de tasa de la API, y asegurando que los datos se almacenen de forma organizada para su posterior análisis.

In [3]:
import numpy as np
import pandas as pd
import requests
import time
import os

In [ ]:
# Cambiar ruta del archivo por la del que queramos procesar
songs_with_data = pd.read_csv('../data/raw/1_nuevos_ids_encontrados.csv')
songs_with_data

,freshmakers,check it and go,21aab354-bfd6-4670-a9e9-599c850b8d48
0,elphomega,stardust,5280281a-e21b-45d5-ae24-e7de6e5cff53
1,ritchie valens,la bamba,86ed1cc6-c0ef-4b27-b026-16d8009d2976
2,falsalarma,plan b,cae6e9aa-123b-48db-ac8c-fb9719d2eeed
3,falsalarma,cuando vuelen nuestras almas,3e8fa044-ed5c-4bda-b294-71ed68b1f485
4,falsalarma,bondad o malicia,7a3bb93d-4f63-4d8e-9ff4-9e5a3a9df602
...,...,...,...
6414,skay beilinson,la grieta,fd8f722a-102b-42cb-9f4d-cf2b823f080d
6415,barricada,el general,ef3914b2-c0d4-459e-a49b-f8655964f80b
6416,alika & nueva alianza,jah proveera,36dce0b1-7aef-437e-984b-9200e56f8768
6417,barricada,de mis párpados se cuelga loko,a1d94d37-9c88-43c0-b43c-887a3d12518e


In [7]:
dataRoute = {
    "danceable" : "['highlevel']['danceability']['all']['danceable']",
    "not_danceable" : "['highlevel']['danceability']['all']['not_danceable']",
    "female" : "['highlevel']['gender']['all']['female']",
    "male" : "['highlevel']['gender']['all']['male']",
    "timbre_bright" : "['highlevel']['timbre']['all']['bright']",
    "timbre_dark" : "['highlevel']['timbre']['all']['dark']",
    "tonal" : "['highlevel']['tonal_atonal']['all']['tonal']",
    "atonal" : "['highlevel']['tonal_atonal']['all']['atonal']",
    "voice" : "['highlevel']['voice_instrumental']['all']['voice']",
    "instrumental" : "['highlevel']['voice_instrumental']['all']['instrumental']",

    "dortmund_alternative" : "['highlevel']['genre_dortmund']['all']['alternative']",
    "dortmund_blues" : "['highlevel']['genre_dortmund']['all']['blues']",
    "dortmund_electronic" : "['highlevel']['genre_dortmund']['all']['electronic']",
    "dortmund_folkcountry" : "['highlevel']['genre_dortmund']['all']['folkcountry']",
    "dortmund_funksoulrnb" : "['highlevel']['genre_dortmund']['all']['funksoulrnb']",
    "dortmund_jazz" : "['highlevel']['genre_dortmund']['all']['jazz']",
    "dortmund_pop" : "['highlevel']['genre_dortmund']['all']['pop']",
    "dortmund_raphiphop" : "['highlevel']['genre_dortmund']['all']['raphiphop']",
    "dortmund_rock" : "['highlevel']['genre_dortmund']['all']['rock']",

    "electronic_ambient" : "['highlevel']['genre_electronic']['all']['ambient']",
    "electronic_dnb" : "['highlevel']['genre_electronic']['all']['dnb']",
    "electronic_house" : "['highlevel']['genre_electronic']['all']['house']",
    "electronic_techno" : "['highlevel']['genre_electronic']['all']['techno']",
    "electronic_trance" : "['highlevel']['genre_electronic']['all']['trance']",

    "rosamerica_cla" : "['highlevel']['genre_rosamerica']['all']['cla']",
    "rosamerica_dan" : "['highlevel']['genre_rosamerica']['all']['dan']",
    "rosamerica_hip" : "['highlevel']['genre_rosamerica']['all']['hip']",
    "rosamerica_jaz" : "['highlevel']['genre_rosamerica']['all']['jaz']",
    "rosamerica_pop" : "['highlevel']['genre_rosamerica']['all']['pop']",
    "rosamerica_rhy" : "['highlevel']['genre_rosamerica']['all']['rhy']",
    "rosamerica_roc" : "['highlevel']['genre_rosamerica']['all']['roc']",
    "rosamerica_spe" : "['highlevel']['genre_rosamerica']['all']['spe']",

    "tzanetakis_blu" : "['highlevel']['genre_tzanetakis']['all']['blu']",
    "tzanetakis_cla" : "['highlevel']['genre_tzanetakis']['all']['cla']",
    "tzanetakis_cou" : "['highlevel']['genre_tzanetakis']['all']['cou']",
    "tzanetakis_dis" : "['highlevel']['genre_tzanetakis']['all']['dis']",
    "tzanetakis_hip" : "['highlevel']['genre_tzanetakis']['all']['hip']",
    "tzanetakis_jaz" : "['highlevel']['genre_tzanetakis']['all']['jaz']",
    "tzanetakis_met" : "['highlevel']['genre_tzanetakis']['all']['met']",
    "tzanetakis_pop" : "['highlevel']['genre_tzanetakis']['all']['pop']",
    "tzanetakis_reg" : "['highlevel']['genre_tzanetakis']['all']['reg']",
    "tzanetakis_roc" : "['highlevel']['genre_tzanetakis']['all']['roc']",

    "ismir04_rhythm_ChaChaCha" : "['highlevel']['ismir04_rhythm']['all']['ChaChaCha']",
    "ismir04_rhythm_Jive" : "['highlevel']['ismir04_rhythm']['all']['Jive']",
    "ismir04_rhythm_Quickstep" : "['highlevel']['ismir04_rhythm']['all']['Quickstep']",
    "ismir04_rhythm_Rumba-American" : "['highlevel']['ismir04_rhythm']['all']['Rumba-American']",
    "ismir04_rhythm_Rumba-International" : "['highlevel']['ismir04_rhythm']['all']['Rumba-International']",
    "ismir04_rhythm_Rumba-Misc" : "['highlevel']['ismir04_rhythm']['all']['Rumba-Misc']",
    "ismir04_rhythm_Samba" : "['highlevel']['ismir04_rhythm']['all']['Samba']",
    "ismir04_rhythm_Tango" : "['highlevel']['ismir04_rhythm']['all']['Tango']",
    "ismir04_rhythm_VienneseWaltz" : "['highlevel']['ismir04_rhythm']['all']['VienneseWaltz']",
    "ismir04_rhythm_Waltz" : "['highlevel']['ismir04_rhythm']['all']['Waltz']",

    "mood_acoustic" : "['highlevel']['mood_acoustic']['all']['acoustic']",
    "mood_not_acoustic" : "['highlevel']['mood_acoustic']['all']['not_acoustic']",
    "mood_aggressive" : "['highlevel']['mood_aggressive']['all']['aggressive']",
    "mood_not_aggressive" : "['highlevel']['mood_aggressive']['all']['not_aggressive']",
    "mood_electronic" : "['highlevel']['mood_electronic']['all']['electronic']",
    "mood_not_electronic" : "['highlevel']['mood_electronic']['all']['not_electronic']",
    "mood_happy" : "['highlevel']['mood_happy']['all']['happy']",
    "mood_not_happy" : "['highlevel']['mood_happy']['all']['not_happy']",
    "mood_party" : "['highlevel']['mood_party']['all']['party']",
    "mood_not_party" : "['highlevel']['mood_party']['all']['not_party']",
    "mood_relaxed" : "['highlevel']['mood_relaxed']['all']['relaxed']",
    "mood_not_relaxed" : "['highlevel']['mood_relaxed']['all']['not_relaxed']",
    "mood_sad" : "['highlevel']['mood_sad']['all']['sad']",
    "mood_not_sad" : "['highlevel']['mood_sad']['all']['not_sad']",
    "mood_mirex1" : "['highlevel']['moods_mirex']['all']['Cluster1']",
    "mood_mirex2" : "['highlevel']['moods_mirex']['all']['Cluster2']",
    "mood_mirex3" : "['highlevel']['moods_mirex']['all']['Cluster3']",
    "mood_mirex4" : "['highlevel']['moods_mirex']['all']['Cluster4']",
    "mood_mirex5" : "['highlevel']['moods_mirex']['all']['Cluster5']",
    "artistsort" : "['metadata']['tags']['artistsort']",
    "genre" : "['metadata']['tags']['genre']",
    "releasecountry" : "['metadata']['tags']['releasecountry']",
    "albumartistsort" : "['metadata']['tags']['albumartistsort']",
    "label" : "['metadata']['tags']['label']",
    "tracknumber" : "['metadata']['tags']['tracknumber']"
}

In [8]:
# Get data from AcousticBrainz by recording_id
def get_data_by_recording_id(recording_id, retries=5, backoff_factor=2):
    url = f"https://acousticbrainz.org/{recording_id}/high-level"
    for attempt in range(retries):
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            wait_time = backoff_factor * (attempt + 1)
            print(f"⚠️ Rate limit exceeded. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        else:
            print(f"⚠️ Error fetching data: {response.status_code}")
            return "Error 404"
    print("⚠️ Max retries exceeded.")
    return None

In [5]:
# Counter variables to monitor the status of requests
total_requests = 0
rows_processed = 0
found_count = 0
not_found_count = 0

In [9]:
# Define los archivos de salida
errors_file = '../data/raw/1_errors_brainz.csv'
processed_file = '../data/raw/1_processed_brainz.csv'

# Crea los archivos iniciales con encabezados si no existen
if not os.path.exists(errors_file):
    pd.DataFrame(columns=['recording_id']).to_csv(errors_file, index=False)
if not os.path.exists(processed_file):
    pd.DataFrame(columns=songs_with_data.columns).to_csv(processed_file, index=False)

In [ ]:
# Apply function for every row in the Dataset
for index in songs_with_data.index:
    recording_id = songs_with_data.loc[index, "recording_id"]

    # Verifica el recording_id
    if not recording_id:
        print(f"  ⚠️ Skipping row {index} - No recording_id found")
        continue

    # Contador para monitorear requests hechas y totales
    total_requests += 1
    rows_processed += 1
    print(f"\n[Progress] Request {total_requests}/{len(songs_with_data)}...")

    # Applicar función para encontrar 
    data = get_data_by_recording_id(recording_id, retries=5, backoff_factor=2)

    # Si no se encuentran datos o el error es un 404
    if not data or data == "Error 404":
        print(f"  ⚠️ Recording_id {recording_id} returned 404 or no data found")
        not_found_count += 1
        error_row = pd.DataFrame({'recording_id': [recording_id]})
        error_row.to_csv(errors_file, mode='a', header=False, index=False)
        continue

    print(f"  ✅ Data found for: {index}")
    found_count += 1

    for key, route in dataRoute.items():
        try:
            value_to_assign = data['highlevel']['danceability']['all']['danceable']
            if not isinstance(value_to_assign, (int, float)):
                value_to_assign = np.nan
            songs_with_data.loc[index, 'danceable'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for danceable")
            songs_with_data.loc[index, 'danceable'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing danceable: {e}")
            songs_with_data.loc[index, 'danceable'] = np.nan
        try:
            value_to_assign = data['highlevel']['danceability']['all'].get('not_danceable', np.nan)
            songs_with_data.loc[index, 'not_danceable'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for not_danceable")
            songs_with_data.loc[index, 'not_danceable'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing not_danceable: {e}")
            songs_with_data.loc[index, 'not_danceable'] = np.nan
        try:
            value_to_assign = data['highlevel']['gender']['all'].get('female', np.nan)
            songs_with_data.loc[index, 'female'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for female")
            songs_with_data.loc[index, 'female'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing female: {e}")
            songs_with_data.loc[index, 'female'] = np.nan
        try:
            value_to_assign = data['highlevel']['gender']['all'].get('male', np.nan)
            songs_with_data.loc[index, 'male'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for male")
            songs_with_data.loc[index, 'male'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing male: {e}")
            songs_with_data.loc[index, 'male'] = np.nan
        try:
            value_to_assign = data['highlevel']['timbre']['all'].get('bright', np.nan)
            songs_with_data.loc[index, 'timbre_bright'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for timbre_bright")
            songs_with_data.loc[index, 'timbre_bright'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing timbre_bright: {e}")
            songs_with_data.loc[index, 'timbre_bright'] = np.nan
        try:
            value_to_assign = data['highlevel']['timbre']['all'].get('dark', np.nan)
            songs_with_data.loc[index, 'timbre_dark'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for timbre_dark")
            songs_with_data.loc[index, 'timbre_dark'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing timbre_dark: {e}")
            songs_with_data.loc[index, 'timbre_dark'] = np.nan
        try:
            value_to_assign = data['highlevel']['tonal_atonal']['all'].get('tonal', np.nan)
            songs_with_data.loc[index, 'tonal'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tonal")
            songs_with_data.loc[index, 'tonal'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tonal: {e}")
            songs_with_data.loc[index, 'tonal'] = np.nan
        try:
            value_to_assign = data['highlevel']['tonal_atonal']['all'].get('atonal', np.nan)
            songs_with_data.loc[index, 'atonal'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for atonal")
            songs_with_data.loc[index, 'atonal'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing atonal: {e}")
            songs_with_data.loc[index, 'atonal'] = np.nan
        try:
            value_to_assign = data['highlevel']['voice_instrumental']['all'].get('voice', np.nan)
            songs_with_data.loc[index, 'voice'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for voice")
            songs_with_data.loc[index, 'voice'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing voice: {e}")
            songs_with_data.loc[index, 'voice'] = np.nan
        try:
            value_to_assign = data['highlevel']['voice_instrumental']['all'].get('instrumental', np.nan)
            songs_with_data.loc[index, 'instrumental'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for instrumental")
            songs_with_data.loc[index, 'instrumental'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing instrumental: {e}")
            songs_with_data.loc[index, 'instrumental'] = np.nan

        try:
            value_to_assign = data['highlevel']['genre_dortmund']['all'].get('alternative', np.nan)
            songs_with_data.loc[index, 'dortmund_alternative'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for dortmund_alternative")
            songs_with_data.loc[index, 'dortmund_alternative'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing dortmund_alternative: {e}")
            songs_with_data.loc[index, 'dortmund_alternative'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_dortmund']['all'].get('blues', np.nan)
            songs_with_data.loc[index, 'dortmund_blues'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for dortmund_blues")
            songs_with_data.loc[index, 'dortmund_blues'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing dortmund_blues: {e}")
            songs_with_data.loc[index, 'dortmund_blues'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_dortmund']['all'].get('electronic', np.nan)
            songs_with_data.loc[index, 'dortmund_electronic'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for dortmund_electronic")
            songs_with_data.loc[index, 'dortmund_electronic'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing dortmund_electronic: {e}")
            songs_with_data.loc[index, 'dortmund_electronic'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_dortmund']['all'].get('folkcountry', np.nan)
            songs_with_data.loc[index, 'dortmund_folkcountry'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for dortmund_folkcountry")
            songs_with_data.loc[index, 'dortmund_folkcountry'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing dortmund_folkcountry: {e}")
            songs_with_data.loc[index, 'dortmund_folkcountry'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_dortmund']['all'].get('funksoulrnb', np.nan)
            songs_with_data.loc[index, 'dortmund_funksoulrnb'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for dortmund_funksoulrnb")
            songs_with_data.loc[index, 'dortmund_funksoulrnb'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing dortmund_funksoulrnb: {e}")
            songs_with_data.loc[index, 'dortmund_funksoulrnb'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_dortmund']['all'].get('jazz', np.nan)
            songs_with_data.loc[index, 'dortmund_jazz'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for dortmund_jazz")
            songs_with_data.loc[index, 'dortmund_jazz'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing dortmund_jazz: {e}")
            songs_with_data.loc[index, 'dortmund_jazz'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_dortmund']['all'].get('pop', np.nan)
            songs_with_data.loc[index, 'dortmund_pop'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for dortmund_pop")
            songs_with_data.loc[index, 'dortmund_pop'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing dortmund_pop: {e}")
            songs_with_data.loc[index, 'dortmund_pop'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_dortmund']['all'].get('raphiphop', np.nan)
            songs_with_data.loc[index, 'dortmund_raphiphop'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for dortmund_raphiphop")
            songs_with_data.loc[index, 'dortmund_raphiphop'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing dortmund_raphiphop: {e}")
            songs_with_data.loc[index, 'dortmund_raphiphop'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_dortmund']['all'].get('rock', np.nan)
            songs_with_data.loc[index, 'dortmund_rock'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for dortmund_rock")
            songs_with_data.loc[index, 'dortmund_rock'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing dortmund_rock: {e}")
            songs_with_data.loc[index, 'dortmund_rock'] = np.nan

        try:
            value_to_assign = data['highlevel']['genre_electronic']['all'].get('ambient', np.nan)
            songs_with_data.loc[index, 'electronic_ambient'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for electronic_ambient")
            songs_with_data.loc[index, 'electronic_ambient'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing electronic_ambient: {e}")
            songs_with_data.loc[index, 'electronic_ambient'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_electronic']['all'].get('dnb', np.nan)
            songs_with_data.loc[index, 'electronic_dnb'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for electronic_dnb")
            songs_with_data.loc[index, 'electronic_dnb'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing electronic_dnb: {e}")
            songs_with_data.loc[index, 'electronic_dnb'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_electronic']['all'].get('house', np.nan)
            songs_with_data.loc[index, 'electronic_house'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for electronic_house")
            songs_with_data.loc[index, 'electronic_house'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing electronic_house: {e}")
            songs_with_data.loc[index, 'electronic_house'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_electronic']['all'].get('techno', np.nan)
            songs_with_data.loc[index, 'electronic_techno'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for electronic_techno")
            songs_with_data.loc[index, 'electronic_techno'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing electronic_techno: {e}")
            songs_with_data.loc[index, 'electronic_techno'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_electronic']['all'].get('trance', np.nan)
            songs_with_data.loc[index, 'electronic_trance'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for electronic_trance")
            songs_with_data.loc[index, 'electronic_trance'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing electronic_trance: {e}")
            songs_with_data.loc[index, 'electronic_trance'] = np.nan

        try:
            value_to_assign = data['highlevel']['genre_rosamerica']['all'].get('cla', np.nan)
            songs_with_data.loc[index, 'rosamerica_cla'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for rosamerica_cla")
            songs_with_data.loc[index, 'rosamerica_cla'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing rosamerica_cla: {e}")
            songs_with_data.loc[index, 'rosamerica_cla'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_rosamerica']['all'].get('dan', np.nan)
            songs_with_data.loc[index, 'rosamerica_dan'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for rosamerica_dan")
            songs_with_data.loc[index, 'rosamerica_dan'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing rosamerica_dan: {e}")
            songs_with_data.loc[index, 'rosamerica_dan'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_rosamerica']['all'].get('hip', np.nan)
            songs_with_data.loc[index, 'rosamerica_hip'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for rosamerica_hip")
            songs_with_data.loc[index, 'rosamerica_hip'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing rosamerica_hip: {e}")
            songs_with_data.loc[index, 'rosamerica_hip'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_rosamerica']['all'].get('jaz', np.nan)
            songs_with_data.loc[index, 'rosamerica_jaz'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for rosamerica_jaz")
            songs_with_data.loc[index, 'rosamerica_jaz'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing rosamerica_jaz: {e}")
            songs_with_data.loc[index, 'rosamerica_jaz'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_rosamerica']['all'].get('pop', np.nan)
            songs_with_data.loc[index, 'rosamerica_pop'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for rosamerica_pop")
            songs_with_data.loc[index, 'rosamerica_pop'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing rosamerica_pop: {e}")
            songs_with_data.loc[index, 'rosamerica_pop'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_rosamerica']['all'].get('rhy', np.nan)
            songs_with_data.loc[index, 'rosamerica_rhy'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for rosamerica_rhy")
            songs_with_data.loc[index, 'rosamerica_rhy'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing rosamerica_rhy: {e}")
            songs_with_data.loc[index, 'rosamerica_rhy'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_rosamerica']['all'].get('roc', np.nan)
            songs_with_data.loc[index, 'rosamerica_roc'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for rosamerica_roc")
            songs_with_data.loc[index, 'rosamerica_roc'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing rosamerica_roc: {e}")
            songs_with_data.loc[index, 'rosamerica_roc'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_rosamerica']['all'].get('spe', np.nan)
            songs_with_data.loc[index, 'rosamerica_spe'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for rosamerica_spe")
            songs_with_data.loc[index, 'rosamerica_spe'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing rosamerica_spe: {e}")
            songs_with_data.loc[index, 'rosamerica_spe'] = np.nan

        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('blu', np.nan)
            songs_with_data.loc[index, 'tzanetakis_blu'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_blu")
            songs_with_data.loc[index, 'tzanetakis_blu'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_blu: {e}")
            songs_with_data.loc[index, 'tzanetakis_blu'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('cla', np.nan)
            songs_with_data.loc[index, 'tzanetakis_cla'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_cla")
            songs_with_data.loc[index, 'tzanetakis_cla'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_cla: {e}")
            songs_with_data.loc[index, 'tzanetakis_cla'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('cou', np.nan)
            songs_with_data.loc[index, 'tzanetakis_cou'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_cou")
            songs_with_data.loc[index, 'tzanetakis_cou'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_cou: {e}")
            songs_with_data.loc[index, 'tzanetakis_cou'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('dis', np.nan)
            songs_with_data.loc[index, 'tzanetakis_dis'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_dis")
            songs_with_data.loc[index, 'tzanetakis_dis'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_dis: {e}")
            songs_with_data.loc[index, 'tzanetakis_dis'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('hip', np.nan)
            songs_with_data.loc[index, 'tzanetakis_hip'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_hip")
            songs_with_data.loc[index, 'tzanetakis_hip'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_hip: {e}")
            songs_with_data.loc[index, 'tzanetakis_hip'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('jaz', np.nan)
            songs_with_data.loc[index, 'tzanetakis_jaz'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_jaz")
            songs_with_data.loc[index, 'tzanetakis_jaz'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_jaz: {e}")
            songs_with_data.loc[index, 'tzanetakis_jaz'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('met', np.nan)
            songs_with_data.loc[index, 'tzanetakis_met'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_met")
            songs_with_data.loc[index, 'tzanetakis_met'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_met: {e}")
            songs_with_data.loc[index, 'tzanetakis_met'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('pop', np.nan)
            songs_with_data.loc[index, 'tzanetakis_pop'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_pop")
            songs_with_data.loc[index, 'tzanetakis_pop'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_pop: {e}")
            songs_with_data.loc[index, 'tzanetakis_pop'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('reg', np.nan)
            songs_with_data.loc[index, 'tzanetakis_reg'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_reg")
            songs_with_data.loc[index, 'tzanetakis_reg'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_reg: {e}")
            songs_with_data.loc[index, 'tzanetakis_reg'] = np.nan
        try:
            value_to_assign = data['highlevel']['genre_tzanetakis']['all'].get('roc', np.nan)
            songs_with_data.loc[index, 'tzanetakis_roc'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tzanetakis_roc")
            songs_with_data.loc[index, 'tzanetakis_roc'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing tzanetakis_roc: {e}")
            songs_with_data.loc[index, 'tzanetakis_roc'] = np.nan

        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('ChaChaCha', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_ChaChaCha'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_ChaChaCha")
            songs_with_data.loc[index, 'ismir04_rhythm_ChaChaCha'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_ChaChaCha: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_ChaChaCha'] = np.nan
        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('Jive', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_Jive'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_Jive")
            songs_with_data.loc[index, 'ismir04_rhythm_Jive'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_Jive: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_Jive'] = np.nan
        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('Quickstep', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_Quickstep'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_Quickstep")
            songs_with_data.loc[index, 'ismir04_rhythm_Quickstep'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_Quickstep: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_Quickstep'] = np.nan
        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('Rumba-American', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_Rumba-American'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_Rumba-American")
            songs_with_data.loc[index, 'ismir04_rhythm_Rumba-American'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_Rumba-American: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_Rumba-American'] = np.nan
        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('Rumba-International', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_Rumba-International'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_Rumba-International")
            songs_with_data.loc[index, 'ismir04_rhythm_Rumba-International'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_Rumba-International: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_Rumba-International'] = np.nan
        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('Rumba-Misc', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_Rumba-Misc'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_Rumba-Misc")
            songs_with_data.loc[index, 'ismir04_rhythm_Rumba-Misc'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_Rumba-Misc: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_Rumba-Misc'] = np.nan
        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('Samba', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_Samba'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_Samba")
            songs_with_data.loc[index, 'ismir04_rhythm_Samba'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_Samba: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_Samba'] = np.nan
        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('Tango', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_Tango'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_Tango")
            songs_with_data.loc[index, 'ismir04_rhythm_Tango'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_Tango: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_Tango'] = np.nan
        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('VienneseWaltz', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_VienneseWaltz'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_VienneseWaltz")
            songs_with_data.loc[index, 'ismir04_rhythm_VienneseWaltz'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_VienneseWaltz: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_VienneseWaltz'] = np.nan
        try:
            value_to_assign = data['highlevel']['ismir04_rhythm']['all'].get('Waltz', np.nan)
            songs_with_data.loc[index, 'ismir04_rhythm_Waltz'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for ismir04_rhythm_Waltz")
            songs_with_data.loc[index, 'ismir04_rhythm_Waltz'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing ismir04_rhythm_Waltz: {e}")
            songs_with_data.loc[index, 'ismir04_rhythm_Waltz'] = np.nan

        try:
            value_to_assign = data['highlevel']['mood_acoustic']['all'].get('acoustic', np.nan)
            songs_with_data.loc[index, 'mood_acoustic'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_acoustic")
            songs_with_data.loc[index, 'mood_acoustic'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_acoustic: {e}")
            songs_with_data.loc[index, 'mood_acoustic'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_acoustic']['all'].get('not_acoustic', np.nan)
            songs_with_data.loc[index, 'mood_not_acoustic'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_not_acoustic")
            songs_with_data.loc[index, 'mood_not_acoustic'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_not_acoustic: {e}")
            songs_with_data.loc[index, 'mood_not_acoustic'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_aggressive']['all'].get('aggressive', np.nan)
            songs_with_data.loc[index, 'mood_aggressive'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_aggressive")
            songs_with_data.loc[index, 'mood_aggressive'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_aggressive: {e}")
            songs_with_data.loc[index, 'mood_aggressive'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_aggressive']['all'].get('not_aggressive', np.nan)
            songs_with_data.loc[index, 'mood_not_aggressive'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_not_aggressive")
            songs_with_data.loc[index, 'mood_not_aggressive'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_not_aggressive: {e}")
            songs_with_data.loc[index, 'mood_not_aggressive'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_electronic']['all'].get('electronic', np.nan)
            songs_with_data.loc[index, 'mood_electronic'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_electronic")
            songs_with_data.loc[index, 'mood_electronic'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_electronic: {e}")
            songs_with_data.loc[index, 'mood_electronic'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_electronic']['all'].get('not_electronic', np.nan)
            songs_with_data.loc[index, 'mood_not_electronic'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_not_electronic")
            songs_with_data.loc[index, 'mood_not_electronic'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_not_electronic: {e}")
            songs_with_data.loc[index, 'mood_not_electronic'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_happy']['all'].get('happy', np.nan)
            songs_with_data.loc[index, 'mood_happy'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_happy")
            songs_with_data.loc[index, 'mood_happy'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_happy: {e}")
            songs_with_data.loc[index, 'mood_happy'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_happy']['all'].get('not_happy', np.nan)
            songs_with_data.loc[index, 'mood_not_happy'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_not_happy")
            songs_with_data.loc[index, 'mood_not_happy'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_not_happy: {e}")
            songs_with_data.loc[index, 'mood_not_happy'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_party']['all'].get('party', np.nan)
            songs_with_data.loc[index, 'mood_party'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_party")
            songs_with_data.loc[index, 'mood_party'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_party: {e}")
            songs_with_data.loc[index, 'mood_party'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_party']['all'].get('not_party', np.nan)
            songs_with_data.loc[index, 'mood_not_party'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_not_party")
            songs_with_data.loc[index, 'mood_not_party'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_not_party: {e}")
            songs_with_data.loc[index, 'mood_not_party'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_relaxed']['all'].get('relaxed', np.nan)
            songs_with_data.loc[index, 'mood_relaxed'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_relaxed")
            songs_with_data.loc[index, 'mood_relaxed'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_relaxed: {e}")
            songs_with_data.loc[index, 'mood_relaxed'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_relaxed']['all'].get('not_relaxed', np.nan)
            songs_with_data.loc[index, 'mood_not_relaxed'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_not_relaxed")
            songs_with_data.loc[index, 'mood_not_relaxed'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_not_relaxed: {e}")
            songs_with_data.loc[index, 'mood_not_relaxed'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_sad']['all'].get('sad', np.nan)
            songs_with_data.loc[index, 'mood_sad'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_sad")
            songs_with_data.loc[index, 'mood_sad'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_sad: {e}")
            songs_with_data.loc[index, 'mood_sad'] = np.nan
        try:
            value_to_assign = data['highlevel']['mood_sad']['all'].get('not_sad', np.nan)
            songs_with_data.loc[index, 'mood_not_sad'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_not_sad")
            songs_with_data.loc[index, 'mood_not_sad'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_not_sad: {e}")
            songs_with_data.loc[index, 'mood_not_sad'] = np.nan

        try:
            value_to_assign = data['highlevel']['moods_mirex']['all'].get('Cluster1', np.nan)
            songs_with_data.loc[index, 'mood_mirex1'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_mirex1")
            songs_with_data.loc[index, 'mood_mirex1'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_mirex1: {e}")
            songs_with_data.loc[index, 'mood_mirex1'] = np.nan
        try:
            value_to_assign = data['highlevel']['moods_mirex']['all'].get('Cluster2', np.nan)
            songs_with_data.loc[index, 'mood_mirex2'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_mirex2")
            songs_with_data.loc[index, 'mood_mirex2'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_mirex2: {e}")
            songs_with_data.loc[index, 'mood_mirex2'] = np.nan
        try:
            value_to_assign = data['highlevel']['moods_mirex']['all'].get('Cluster3', np.nan)
            songs_with_data.loc[index, 'mood_mirex3'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_mirex3")
            songs_with_data.loc[index, 'mood_mirex3'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_mirex3: {e}")
            songs_with_data.loc[index, 'mood_mirex3'] = np.nan
        try:
            value_to_assign = data['highlevel']['moods_mirex']['all'].get('Cluster4', np.nan)
            songs_with_data.loc[index, 'mood_mirex4'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_mirex4")
            songs_with_data.loc[index, 'mood_mirex4'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_mirex4: {e}")
            songs_with_data.loc[index, 'mood_mirex4'] = np.nan
        try:
            value_to_assign = data['highlevel']['moods_mirex']['all'].get('Cluster5', np.nan)
            songs_with_data.loc[index, 'mood_mirex5'] = round(value_to_assign, 3) if isinstance(value_to_assign, (int, float)) else value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for mood_mirex5")
            songs_with_data.loc[index, 'mood_mirex5'] = np.nan
        except Exception as e:
            print(f"⚠️ Error processing mood_mirex5: {e}")
            songs_with_data.loc[index, 'mood_mirex5'] = np.nan

        try:
            value_to_assign = data['metadata']['tags'].get('artistsort', None)
            songs_with_data.loc[index, 'artistsort'] = value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for artistsort")
            songs_with_data.loc[index, 'artistsort'] = None
        except Exception as e:
            print(f"⚠️ Error processing artistsort: {e}")
            songs_with_data.loc[index, 'artistsort'] = None
        try:
            value_to_assign = data['metadata']['tags'].get('genre', None)
            songs_with_data.loc[index, 'genre'] = value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for genre")
            songs_with_data.loc[index, 'genre'] = None
        except Exception as e:
            print(f"⚠️ Error processing genre: {e}")
            songs_with_data.loc[index, 'genre'] = None
        try:
            value_to_assign = data['metadata']['tags'].get('releasecountry', None)
            songs_with_data.loc[index, 'releasecountry'] = value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for releasecountry")
            songs_with_data.loc[index, 'releasecountry'] = None
        except Exception as e:
            print(f"⚠️ Error processing releasecountry: {e}")
            songs_with_data.loc[index, 'releasecountry'] = None
        try:
            value_to_assign = data['metadata']['tags'].get('albumartistsort', None)
            songs_with_data.loc[index, 'albumartistsort'] = value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for albumartistsort")
            songs_with_data.loc[index, 'albumartistsort'] = None
        except Exception as e:
            print(f"⚠️ Error processing albumartistsort: {e}")
            songs_with_data.loc[index, 'albumartistsort'] = None
        try:
            value_to_assign = data['metadata']['tags'].get('label', None)
            songs_with_data.loc[index, 'label'] = value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for label")
            songs_with_data.loc[index, 'label'] = None
        except Exception as e:
            print(f"⚠️ Error processing label: {e}")
            songs_with_data.loc[index, 'label'] = None
        try:
            value_to_assign = data['metadata']['tags'].get('tracknumber', None)
            songs_with_data.loc[index, 'tracknumber'] = value_to_assign
        except KeyError:
            print(f"⚠️ Missing key for tracknumber")
            songs_with_data.loc[index, 'tracknumber'] = None
        except Exception as e:
            print(f"⚠️ Error processing tracknumber: {e}")
            songs_with_data.loc[index, 'tracknumber'] = None

    
    # Escribe la fila procesada en el archivo correspondiente
    processed_row = songs_with_data.iloc[[index]]  # Selecciona la fila procesada
    processed_row.to_csv(processed_file, mode='a', header=False, index=False)

    # Print percentage of "Not found" vs valid IDs
    total_processed_so_far = found_count + not_found_count
    total_rows = len(songs_with_data)
    print(f"  [Stats] - Not Found: {not_found_count} Found: {found_count}")
    print(f"  [Stats] - Processed: {total_processed_so_far} / {total_rows}")

    time.sleep(3)

In [ ]:
songs = pd.read_csv(r'C:\Users\solan\Downloads\get_data_from_songs\data\raw\4_processed_brainz.csv')
songs.info()

In [59]:
# Seleccionar los primeros 2950 registros para guardarlos en un .csv a parte, ya que son los que se han procesado
# Revisar dataset .temp y cambiar el número por el que sea necesario según se haya quedado el proceso en 
processed_data_songs = songs.head(1)

# Guardar los datos procesados en un nuevo archivo CSV
processed_data_songs.to_csv('processed_data_songs.csv', index=False)

In [60]:
processed_data_songs

,artist_name,song_name,recording_id,error,danceable,not_danceable,female,male,timbre_bright,timbre_dark,...,mood_mirex2,mood_mirex3,mood_mirex4,mood_mirex5,artistsort,genre,releasecountry,albumartistsort,label,tracknumber
0,the stone roses,sally cinnamon,cd862f18-6a1c-4ad9-8f94-4978ab102bc7,404.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,the stone roses,here it comes,4bf61c3e-dce8-4d7a-a13c-a54c5efc4920,404.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,the stone roses,all across the sands,03d48ecf-ae27-47f2-bad7-26f787606bd9,NaN,0.938,0.062,0.005,0.995,0.986,0.014,...,0.273,0.433,0.133,0.048,"['Stone Roses, The']",['Indie Rock'],['GB'],"['Stone Roses, The']",['Silvertone Records'],['5']
3,the stone roses,elephant stone,ba0be05d-65c5-49e7-aab7-2279adcd8b90,NaN,0.007,0.993,0.516,0.484,0.345,0.655,...,0.074,0.029,0.037,0.658,"['Stone Roses, The']",['Indie Rock'],['GB'],"['Stone Roses, The']",['Silvertone Records'],['6']
4,the stone roses,the hardest thing in the world,5fb289c0-2be5-49b8-9e8f-23cf12c60ce8,NaN,0.672,0.328,0.085,0.915,0.988,0.012,...,0.119,0.450,0.141,0.179,"['Stone Roses, The']",['Indie Rock'],['GB'],"['Stone Roses, The']",['Silvertone Records'],['4']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,cody simpson,iyiyi,f502acf3-e0d3-4bb4-bc7e-52a333631507,404.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
446,cody simpson,all day,b2fac1d1-e744-481d-8a8a-838b6c42c2fb,404.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,cody simpson,good as it gets,b94e763f-e6b0-4e61-8e24-4fd9f16b63cb,NaN,0.811,0.189,0.711,0.289,0.912,0.088,...,0.306,0.081,0.262,0.211,NaN,['流行'],NaN,NaN,NaN,['01']
448,cody simpson,crazy but true,244e9c68-c092-45c4-a907-352abb1bf3ba,NaN,0.996,0.004,0.980,0.020,0.877,0.123,...,0.170,0.086,0.138,0.488,NaN,['流行'],NaN,NaN,NaN,['02']


In [61]:
# Eliminar los primeros 2950 registros del dataset de canciones que aún no hemos procesado, así eliminamos las que ya han pasado y nos ahorramos recorrerla entera de nuevo
# Revisar dataset .temp y cambiar el número por el que sea necesario según se haya quedado el proceso en 
remaining_data = songs.iloc[450:]

# Guardar los datos restantes en el archivo original o en un nuevo archivo
# Tras hacer esto, cambiamos la ruta del csv de la segunda celda de este archivo a esta
remaining_data.to_csv('not_processed_songs.csv', index=False)

In [63]:
data = pd.read_csv('not_processed_songs.csv')

,artist_name,song_name,recording_id,error,danceable,not_danceable,female,male,timbre_bright,timbre_dark,...,mood_mirex2,mood_mirex3,mood_mirex4,mood_mirex5,artistsort,genre,releasecountry,albumartistsort,label,tracknumber
0,slayyyter,mine,36a6b97c-59df-4a4f-a654-61919b78f888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,slayyyter,alone,89b8745d-ed0d-4b10-a267-32879b3a0c56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,slim thug,tha boss,787511cf-c770-4639-a957-a13edb2b5f50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,slim thug,the bossman,d380d29b-6940-45c5-8fcb-0d45b368eb6b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,slim thug,got plex,c882e5db-9382-4682-8226-0775b9e8d6a4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20820,michael w. smith,missing person,7041f123-f0d6-4db0-ac8c-a9ff5314f194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20821,michael w. smith,love me good,8ad3921f-3baa-4f4d-842c-37d6ae29c2d8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20822,michael w. smith,live the life,ba1466c4-ce37-47c8-b408-76e4500c4b21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20823,michael w. smith,never been unloved,cd5dca5d-907b-4bb1-b22f-28fc8c29b7bf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
